# Лабораторная работа 4. Полносвязные нейронные сети (многослойный персептрон). Решение задач регрессии и классификации

# Задание
 <b>Традиционное предупреждение для всех лабораторных работ:</b> перед обучением моделей необходимо выполнить предварительную обработку данных, которая <b>обязательно</b> включает в себя:
- заполнение пропущенных значений (рекомедуется логика заполнения пропусков на основе типа данных, которая использовалась в РГР по Практикуму);
- преобразование категориальных признаков в числовые (используйте one-hot кодирование или map; используйте знания с Практикума).

<b>Выполните следующие задания:</b>
- решите задачи регрессии и классификации на ваших данных используя полносвязные нейронные сети; соберите их используя API Keras фреймворка TensorFlow; оцените качество полученных моделей с помощью метрик; 
- реализуйте многослойный персептрон, с помощью которого можно решать задачи регрессии и классификации; предусмотрите возможность использовать такие функции активации, как sigmoid, tanh и relu; также предусмотрите возможность указать, сколько слоев нужно, сколько на каждом из них нейронов и какую функцию активации должен иметь слой; реализуйте обучение персептрона методом обратного распространения ошибки; самостоятельно найдите производные функций sigmoid, tanh и relu; реализуйте классический градиентный спуск с возможностью указания шага.

<b>Дополнительные задания:</b>
- самостоятельно изучите отличия работы оптимизаторов Adam и RMSProp от классического градиентного спуска; реализуйте градиентный спуск с использованием указанных оптимизаторов; предусмотрите возможность использования реализованных вами оптимизаторов в вашем персептроне.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, max_error, mean_squared_error, r2_score, confusion_matrix, classification_report
import tensorflow as tf
import numpy as np
from math import sqrt

**Загрузим предобработанные в прошлых лр датасеты**

In [2]:
data_reg = pd.read_csv('../data/mumbai.csv')
data_сlr = pd.read_csv('..\data\smoke_detector.csv')

In [3]:
data_сlr

,Unnamed: 0,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,Fire Alarm
0,50065,23.41,69.47,43.0,1120.0,13105.0,19890.0,931.239,14299.14,45432.26,21592.18,51914.680,30026.438,1.0
1,50064,23.00,70.28,50.0,1137.0,13094.0,19887.0,931.242,14333.69,45018.38,22959.95,51413.852,29590.688,1.0
2,50066,23.66,68.41,52.0,1141.0,13100.0,19882.0,931.251,14149.05,44786.43,21789.97,51167.711,29547.281,1.0
3,50124,25.86,62.03,546.0,1777.0,12872.0,19414.0,931.213,14318.17,44357.90,24472.12,50626.562,28964.852,1.0
4,50123,25.74,62.93,542.0,1710.0,12878.0,19426.0,931.215,14205.93,43765.07,24896.17,49936.551,28499.859,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62625,25001,13.00,53.20,0.0,400.0,12439.0,19114.0,939.758,0.00,0.00,0.00,0.000,0.000,0.0
62626,25002,13.06,52.81,0.0,400.0,12448.0,19155.0,939.758,0.00,0.00,0.00,0.000,0.000,0.0
62627,25142,11.81,50.49,31.0,445.0,12545.0,19632.0,939.810,0.00,0.00,0.00,0.001,0.000,0.0
62628,25143,11.69,51.08,28.0,453.0,12548.0,19631.0,939.818,0.00,0.00,0.00,0.001,0.000,0.0


In [4]:
data_reg

,Unnamed: 0,price,area,latitude,longitude,bedrooms,bathrooms,balcony,status,neworold,parking,furnished_status,lift,type_of_building_individual house
0,0,22400000.0,629.0,19.032800,72.896357,2.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1,35000000.0,974.0,19.032800,72.896357,3.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,2,31700000.0,968.0,19.085600,72.909277,3.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,3,18700000.0,629.0,19.155756,72.846862,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,0.0
4,4,13500000.0,1090.0,19.177555,72.849887,2.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6233,6250,19500000.0,810.0,19.138320,72.810020,2.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6234,6251,22000000.0,1400.0,19.221920,72.854250,3.0,3.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
6235,6252,20000000.0,750.0,19.144320,72.824111,2.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6236,6253,11000000.0,700.0,19.047201,72.872225,2.0,2.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0


## Предобработка данных таблицы для регрессии

In [5]:
# 1. Выполним анализ столбцов таблицы для регрессии.
# 1.1 Посмотрим список всех столбцов таблицы. Удалим из таблицы поля,
#содержащие уникальные значения (похожие на id) и малоинформативные

data_reg.drop(columns =['Unnamed: 0'], axis=1, inplace=True)
data_reg

,price,area,latitude,longitude,bedrooms,bathrooms,balcony,status,neworold,parking,furnished_status,lift,type_of_building_individual house
0,22400000.0,629.0,19.032800,72.896357,2.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,35000000.0,974.0,19.032800,72.896357,3.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,31700000.0,968.0,19.085600,72.909277,3.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,18700000.0,629.0,19.155756,72.846862,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,0.0
4,13500000.0,1090.0,19.177555,72.849887,2.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6233,19500000.0,810.0,19.138320,72.810020,2.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6234,22000000.0,1400.0,19.221920,72.854250,3.0,3.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
6235,20000000.0,750.0,19.144320,72.824111,2.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6236,11000000.0,700.0,19.047201,72.872225,2.0,2.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0


In [6]:
data_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6238 entries, 0 to 6237
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   price                              6238 non-null   float64
 1   area                               6238 non-null   float64
 2   latitude                           6238 non-null   float64
 3   longitude                          6238 non-null   float64
 4   bedrooms                           6238 non-null   float64
 5   bathrooms                          6238 non-null   float64
 6   balcony                            6238 non-null   float64
 7   status                             6238 non-null   float64
 8   neworold                           6238 non-null   float64
 9   parking                            6238 non-null   float64
 10  furnished_status                   6238 non-null   float64
 11  lift                               6238 non-null   float

### Выделение целевого признака и предикторов

In [7]:
y_reg = data_reg["price"]  #целевой признак
X_reg = data_reg.drop(["price"], axis=1) #предикторы

### Разбиение данных на обучающую и тестовую выборки

In [8]:
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.1, random_state=59)

In [9]:
#для удобства создадим функцию для метрик регрессии
def metrics(y_test, y_pred):
    print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
    print(f'MSE: {mean_squared_error(y_test, y_pred)}')
    print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
    print(f'R^2: {r2_score(y_test, y_pred)}')

### Предобработка данных таблицы для классификации

In [10]:
data_сlr.drop(columns =['Unnamed: 0'], axis=1, inplace=True)
data_сlr

,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,Fire Alarm
0,23.41,69.47,43.0,1120.0,13105.0,19890.0,931.239,14299.14,45432.26,21592.18,51914.680,30026.438,1.0
1,23.00,70.28,50.0,1137.0,13094.0,19887.0,931.242,14333.69,45018.38,22959.95,51413.852,29590.688,1.0
2,23.66,68.41,52.0,1141.0,13100.0,19882.0,931.251,14149.05,44786.43,21789.97,51167.711,29547.281,1.0
3,25.86,62.03,546.0,1777.0,12872.0,19414.0,931.213,14318.17,44357.90,24472.12,50626.562,28964.852,1.0
4,25.74,62.93,542.0,1710.0,12878.0,19426.0,931.215,14205.93,43765.07,24896.17,49936.551,28499.859,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62625,13.00,53.20,0.0,400.0,12439.0,19114.0,939.758,0.00,0.00,0.00,0.000,0.000,0.0
62626,13.06,52.81,0.0,400.0,12448.0,19155.0,939.758,0.00,0.00,0.00,0.000,0.000,0.0
62627,11.81,50.49,31.0,445.0,12545.0,19632.0,939.810,0.00,0.00,0.00,0.001,0.000,0.0
62628,11.69,51.08,28.0,453.0,12548.0,19631.0,939.818,0.00,0.00,0.00,0.001,0.000,0.0


### Выделение целевого признака и предикторов

In [11]:
y_clr = data_сlr["Fire Alarm"]  #целевой признак
X_clr = data_сlr.drop(["Fire Alarm"], axis=1) #предикторы

### Балансировка для задачи классификации

In [12]:
#!pip install imbalanced-learn
# 1 - мажоритарный класс (доля примеров слишком высокая)
# 0 - миноритарный класс (доля примеров слишком низкая)
from imblearn.under_sampling import RandomUnderSampler #случайное удаление
#вычисляем количество экземпляров мажоритарного класса, которые нужно удалить
print(f"Количество объектов каждого класса до under_sampling: \n{y_clr.value_counts()}")
underSampler = RandomUnderSampler(sampling_strategy='majority') # пересэмплировать только мажоритарный класс
# удалим некоторое число примеров мажоритарного (большего) класса
X_under_sample, y_under_sample = underSampler.fit_resample(X_clr, y_clr)# повторить выборку 
print(f"Количество объектов каждого класса после under_sampling: \n{y_under_sample.value_counts()}")

Количество объектов каждого класса до under_sampling: 
Fire Alarm
1.0    44757
0.0    17873
Name: count, dtype: int64
Количество объектов каждого класса после under_sampling: 
Fire Alarm
0.0    17873
1.0    17873
Name: count, dtype: int64


### Разбиение данных на обучающую и тестовую выборки

In [13]:
X_clr_train, X_clr_test, y_clr_train, y_clr_test = train_test_split(X_under_sample, y_under_sample, test_size=0.33, random_state=42)
X_clr_train.shape, y_clr_train.shape, X_clr_test.shape, y_clr_test.shape

((23949, 12), (23949,), (11797, 12), (11797,))

In [14]:
y_clr_test

8413     0.0
8525     0.0
19214    1.0
20721    1.0
14573    0.0
        ... 
14667    0.0
1429     0.0
29746    1.0
9048     0.0
10924    0.0
Name: Fire Alarm, Length: 11797, dtype: float64

### Масштабирование (изменение диапазона) числовых признаков 

In [15]:
#Масштабирование - процесс повышения пригодности функций для обучения путем изменения масштаба

#Это необходимый шаг, потому что в большинстве случаев числовые характеристики не имеют определенного диапазона и отличаются друг от друга. 
#Это сильно искажает результаты таких алгоритмов, как метод k-ближайших соседей, которые учитывают расстояния между измерениями.
#А масштабирование позволяет этого избежать. 


**Нормализация**: все значения будут находиться в диапазоне от 0 до 1

Для того, чтобы сделать нормализацию данных нужно посчитать в каждом признаке его минимум (min) и максимум (max), а затем сделать следующее вычисление:

$$x = \frac{x - min}{max - min}$$

После такого преобразования $min = 0, max = 1$.

**Стандартизация**: масштабирует значения с учетом стандартного отклонения.

Для того, чтобы сделать стандартизацию данных нужно посчитать в каждом признаке его среднее значение (`mean`) и стандартное отклонение (`std`), а затем сделать следующее вычисление:

$$x = \frac{x - mean}{std}$$

После такого преобразования $mean = 0, std = 1$.

In [16]:
#нормализация

mms = MinMaxScaler() #инициализируем модель
mms.fit(X_clr_train) #обучаем на обучающей выборке (расчет min и max)

MinMaxScaler()

In [17]:
#нормализируем нашу выборку
#для каждого значения вычтем его минимум и поделим на разницу максимума и минимума
X_train_norms = mms.transform(X_clr_train) #transform возвращает вложенные вычисления в каждой группе
X_test_norms = mms.transform(X_clr_test)
X_train_norms

array([[5.67854528e-01, 6.65063605e-01, 2.25666667e-02, ...,
        1.92576595e-04, 3.55583430e-05, 1.39876731e-06],
       [5.72003905e-01, 6.38070121e-01, 1.90833333e-02, ...,
        2.38931603e-04, 4.41300996e-05, 1.73180715e-06],
       [5.21845253e-01, 6.22246354e-01, 4.33333333e-03, ...,
        6.84752927e-05, 1.26553799e-05, 4.99559755e-07],
       ...,
       [4.36795216e-01, 6.44896060e-01, 2.15000000e-03, ...,
        1.06209896e-04, 1.96283595e-05, 7.65991624e-07],
       [2.92897242e-04, 6.20539870e-03, 1.00000000e+00, ...,
        9.24823074e-03, 1.91672182e-03, 2.89411618e-04],
       [5.78899195e-01, 6.13403661e-01, 0.00000000e+00, ...,
        5.51380623e-05, 1.01897960e-05, 3.99647804e-07]])

In [18]:
X_clr_train = pd.DataFrame(mms.transform(X_clr_train), columns=X_clr_train.columns)
X_clr_test = pd.DataFrame(mms.transform(X_clr_test), columns=X_clr_test.columns)
X_clr_train

,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5
0,0.567855,0.665064,0.022567,0.000144,0.735885,0.668472,0.872350,0.000120,0.000039,0.000193,0.000036,1.398767e-06
1,0.572004,0.638070,0.019083,0.003091,0.692504,0.677499,0.892219,0.000149,0.000049,0.000239,0.000044,1.731807e-06
2,0.521845,0.622246,0.004333,0.000000,0.787241,0.769572,0.957487,0.000043,0.000014,0.000068,0.000013,4.995598e-07
3,0.500659,0.583618,0.003000,0.000815,0.670494,0.858198,0.730492,0.000089,0.000029,0.000143,0.000026,1.032423e-06
4,0.248145,0.582687,0.002517,0.000000,0.810207,0.779583,0.974803,0.000122,0.000040,0.000194,0.000036,1.398767e-06
...,...,...,...,...,...,...,...,...,...,...,...,...
23944,0.461801,0.607819,0.000033,0.000000,0.749282,0.757591,0.987124,0.000022,0.000007,0.000034,0.000006,2.331279e-07
23945,0.171064,0.467732,0.000917,0.000000,0.678150,0.878385,0.734377,0.000129,0.000042,0.000206,0.000038,1.498679e-06
23946,0.436795,0.644896,0.002150,0.000000,0.798405,0.776465,0.977245,0.000066,0.000022,0.000106,0.000020,7.659916e-07
23947,0.000293,0.006205,1.000000,0.025780,0.505582,0.401773,0.651349,0.005952,0.002097,0.009248,0.001917,2.894116e-04


## Использование фреймворка TensorFlow и API Keras для построения нейронных сетей

### Регрессия

In [19]:
# создаем модель, как набор последовательных слоев
model_regression = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=(12,)),
        # Dense - полносвязный слой (каждый нейрон следующего слоя связан со всеми нейронами предыдущего)
        tf.keras.layers.Dense(256, activation="relu"),
        # на втором скрытом слое будет 512 нейронов
        tf.keras.layers.Dense(512, activation="linear"),
        # Dropout позволяет внести фактор случайности - при обучении часть нейронов будет отключаться
        # каждый нейрон, в данном случае, будет отключаться с вероятностью 0.1
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.01),
        # на выходе один нейрон, функция активации не применяется
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [20]:
# посмотрим, какая сеть у нас получилась
model_regression.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               3328      
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               65664     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0

Видим количество обучаемых параметров каждого слоя и общее количество обучаемых параметров. Перед использованием модель необходимо скомпилировать, при этом указывается оптимизатор, скорость обучения (можно представлять как величину шага в методе градиентного спуска), функция потерь и метрики, которые мы хотим (при желании) вычислять в будущем методом evaluate().

In [21]:
# компилируем
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mae")

In [22]:
# обучаем, 20 эпох означает 20 проходов по обучающей выборке
model_regression.fit(X_reg_train, y_reg_train, epochs=20)

Epoch 1/20
176/176 [==============================] - 2s 6ms/step - loss: 13953963.0000
Epoch 2/20
176/176 [==============================] - 1s 6ms/step - loss: 11730853.0000
Epoch 3/20
176/176 [==============================] - 1s 6ms/step - loss: 11824670.0000
Epoch 4/20
176/176 [==============================] - 1s 6ms/step - loss: 11534771.0000
Epoch 5/20
176/176 [==============================] - 1s 6ms/step - loss: 11593098.0000
Epoch 6/20
176/176 [==============================] - 1s 6ms/step - loss: 11588601.0000
Epoch 7/20
176/176 [==============================] - 1s 6ms/step - loss: 11657884.0000
Epoch 8/20
176/176 [==============================] - 1s 6ms/step - loss: 11630204.0000
Epoch 9/20
176/176 [==============================] - 1s 6ms/step - loss: 11660039.0000
Epoch 10/20
176/176 [==============================] - 1s 6ms/step - loss: 11435863.0000
Epoch 11/20
176/176 [==============================] - 1s 6ms/step - loss: 11469676.0000
Epoch 12/20
176/176 [=========

In [23]:
y_reg_pred = model_regression.predict(X_reg_test)

20/20 [==============================] - 0s 4ms/step


In [24]:
metrics(y_reg_test, y_reg_pred)

MAE: 9917011.797275642
MSE: 266422199956712.0
RMSE: 16322444.668514334
MAPE: 0.752913049618333
R^2: 0.6932130976359524


### Классификация

Нейронная сеть для решения задачи классификации будет очень похожа на ту сеть для регрессии, однако у нее по другому будет организован выходной слой. У нас есть 2 стратегии наполнения выходного слоя нейронами:

- при решении задачи бинарной классификации мы можем расположить на выходном слое один нейрон с функцией активации sigmoid (значения от 0 и 1), после чего округлять полученные значения; значение нейрона покажет уверенность сети в предсказании; также мы можем расположить 2 нейрона на выходном слое и применить функцию softmax. Тогда сумма значений нейронов выходного слоя будет 1, а предсказание мы сможем получить определив нейрон с наибольшим значением;
- в случае многоклассовой классификации, как правило, на выходном слое располагаются k нейронов (по количеству классов), функция активации - softmax; нейрон с наибольшим значением определяет предсказанный класс.

In [25]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=(12,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(256, activation="relu"),
        #tf.keras.layers.Dense(1024, activation="tanh"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(512, activation="tanh"),
        tf.keras.layers.Dropout(0.01),
        tf.keras.layers.Dense(128, activation="tanh"),
        tf.keras.layers.Dense(64, activation="tanh"),
        tf.keras.layers.Dense(32, activation="tanh"),
        tf.keras.layers.Dense(16, activation="tanh"),
        # сначала используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

In [26]:
# в качестве функции активации используется бинарная  кроссэнтропия
model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="binary_crossentropy")
# verbose=None - не будет логов
model_classification_1.fit(X_clr_train, y_clr_train, epochs=25)

Epoch 1/25
749/749 [==============================] - 6s 7ms/step - loss: 0.2041
Epoch 2/25
749/749 [==============================] - 5s 7ms/step - loss: 0.1224
Epoch 3/25
749/749 [==============================] - 5s 7ms/step - loss: 0.1065
Epoch 4/25
749/749 [==============================] - 5s 7ms/step - loss: 0.0950
Epoch 5/25
749/749 [==============================] - 5s 7ms/step - loss: 0.0920
Epoch 6/25
749/749 [==============================] - 5s 7ms/step - loss: 0.0901
Epoch 7/25
749/749 [==============================] - 2s 3ms/step - loss: 0.0777
Epoch 8/25
749/749 [==============================] - 2s 3ms/step - loss: 0.0719
Epoch 9/25
749/749 [==============================] - 2s 2ms/step - loss: 0.0623
Epoch 10/25
749/749 [==============================] - 2s 2ms/step - loss: 0.0601
Epoch 11/25
749/749 [==============================] - 2s 2ms/step - loss: 0.0630
Epoch 12/25
749/749 [==============================] - 2s 2ms/step - loss: 0.0599
Epoch 13/25
749/749 [====

In [27]:
model_classification_1.predict(X_clr_test, verbose=None)

array([[0.00235275],
       [0.00235277],
       [0.9978314 ],
       ...,
       [0.9978314 ],
       [0.00235277],
       [0.00302163]], dtype=float32)

In [28]:
y_clr1_pred = np.around(model_classification_1.predict(X_clr_test, verbose=None))
print(classification_report(y_clr_test, y_clr1_pred))
print(confusion_matrix(y_clr_test, y_clr1_pred))

              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99      5884
         1.0       0.98      1.00      0.99      5913

    accuracy                           0.99     11797
   macro avg       0.99      0.99      0.99     11797
weighted avg       0.99      0.99      0.99     11797

[[5784  100]
 [   9 5904]]


In [30]:
model_classification_1.save('C:/Users/verab/Desktop/save/Neuro_classification_model')

INFO:tensorflow:Assets written to: C:/Users/verab/Desktop/save/Neuro_classification_model\assets


INFO:tensorflow:Assets written to: C:/Users/verab/Desktop/save/Neuro_classification_model\assets


## Реализация многослойного персептрона

Реализуйте многослойный персептрон, с помощью которого можно решать задачи регрессии и классификации; предусмотрите возможность использовать такие функции активации, как sigmoid, tanh и relu; также предусмотрите возможность указать, сколько слоев нужно, сколько на каждом из них нейронов и какую функцию активации должен иметь слой; реализуйте обучение персептрона методом обратного распространения ошибки; самостоятельно найдите производные функций sigmoid, tanh и relu; реализуйте классический градиентный спуск с возможностью указания шага.

In [ ]:
df = pd.read_csv('../data/mumbai.csv')
y = df['price']
X = df.drop(['price'], axis=1)
X.reset_index(drop= True , inplace= True)
sc = MinMaxScaler()
X_scaled = sc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)
X_train

In [ ]:
class Sigma:

    def __init__(self, layer_sizes, activations):
        self.layer_sizes = layer_sizes
        self.activations = activations
        self.weights = []
        self.biases = []
        self.num_layers = len(layer_sizes)
        
        #инициализируем веса и смещения
        for i in range(1, self.num_layers):
            self.weights.append(np.random.randn(layer_sizes[i], layer_sizes[i-1]))
            self.biases.append(np.random.randn(layer_sizes[i], 1))
    
    #Понижаю размерность двигаясь вперед (происходит перемножением матриц)
    def forward_propagation(self, X):
        activations = [X.T]
        zs = []
        
        for i in range(self.num_layers - 1):
            z = np.dot(self.weights[i], activations[i]) + self.biases[i]
            zs.append(z)
            activations.append(self.activation_function(z, self.activations[i]))
        
        return activations, zs
    
    #Функции активации
    def activation_function(self, z, activation):
        # Сигмоидная функция активации: f(x) = 1 / (1 + e^(-x))
        if activation == 'sigmoid': 
            return 1 / (1 + np.exp(-z))
        #Гиперболический тангенс: f(x) = (2 / (1 + e^(-2x)))-1
        #2*sigmoid(2x)-1 скорректированная сигмоидная функция
        elif activation == 'tanh':
            return np.tanh(z)
        #ReLu возвращает значение х, если х положительно, и 0 в противном случае
        elif activation == 'relu':
            return np.maximum(0, z)
    
    #поизводная функции активации
    def activation_derivative(self, z, activation):
        if activation == 'sigmoid':
            sigmoid = self.activation_function(z, 'sigmoid')
            return sigmoid * (1 - sigmoid)
        elif activation == 'tanh':
            return 1 - np.tanh(z) ** 2
        elif activation == 'relu':
            return np.where(z > 0, 1, 0)
    
    #Двигаюсь назад, пересчитываем веса и повышаем размерность
    def backward_propagation(self, X, y, activations, zs):
        m = X.shape[0]
        deltas = [None] * self.num_layers
        
        deltas[-1] = activations[-1] - y.T
        for i in range(self.num_layers - 2, 0, -1):
            deltas[i] = np.dot(self.weights[i].T, deltas[i+1]) * self.activation_derivative(zs[i-1], self.activations[i-1])
        
        grad_weights = []
        grad_biases = []
        
        for i in range(self.num_layers - 1):
            grad_weights.append((1/m) * np.dot(deltas[i+1], activations[i].T))
            grad_biases.append((1/m) * np.sum(deltas[i+1], axis=1, keepdims=True))
        
        return grad_weights, grad_biases

    #обучаем
    def fit(self, X, y, epochs, learning_rate=0.1):
        ynp = np.array(y)
        for iteration in range(epochs):
            activations, zs = self.forward_propagation(X)
            grad_weights, grad_biases = self.backward_propagation(X, ynp, activations, zs)
            
            for i in range(self.num_layers - 1):
                self.weights[i] -= learning_rate * grad_weights[i]
                self.biases[i] -= learning_rate * grad_biases[i]

    #предсказываем
    def predict(self, X):
        activations, _ = self.forward_propagation(X)
        return activations[-1].T

In [ ]:
sig = Sigma(layer_sizes=[13, 10, 5, 1], activations=['relu', 'tanh','relu'])

In [ ]:
sig.fit(X_train, y_train, epochs=250, learning_rate=0.1)

In [ ]:
y_pred = sig.predict(X_test)
y_pred

In [ ]:
metrics(y_test, y_pred)